# TAEG (Temporal Alignment Event Graph) - Data Exploration

This notebook implements a complete exploratory data analysis for the TAEG project for abstractive multi-document summarization using Graph Neural Networks (GNNs) and Natural Language Processing (NLP).

## Objective
Explore and analyze the five XML files that form the basis of the TAEG system:
- ChronologyOfTheFourGospels_PW.xml (event structure)
- EnglishNIVMatthew40_PW.xml, EnglishNIVMark41_PW.xml, EnglishNIVLuke42_PW.xml, EnglishNIVJohn43_PW.xml (gospel texts)

## Notebook Structure
1. **Data Loading and XML Parsing** - Loading and parsing XMLs
2. **Graph Construction from Chronology** - Temporal graph construction 
3. **Node Content Extraction and Alignment** - Node content extraction
4. **Temporal Edge Creation** - Temporal edge creation
5. **TAEG Model Implementation** - TAEG model implementation
6. **Baseline Models Setup** - Baseline models configuration
7. **Training Pipeline** - Training pipeline
8. **Evaluation and Metrics** - Evaluation and metrics

In [ ]:
# Imports and configuration
import sys
import os
from pathlib import Path
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import networkx as nx
from collections import defaultdict, Counter
import warnings
warnings.filterwarnings('ignore')

# Add src directory to path for imports
project_root = Path.cwd().parent
sys.path.append(str(project_root / "src"))

# Configure plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

print("✅ Libraries imported successfully")
print(f"📁 Project directory: {project_root}")
print(f"📁 Data directory: {project_root / 'data'}")

## 2. Data Loading and XML Parsing

Loading and parsing the XML files that contain the chronology and gospel texts.

In [ ]:
# Initialize TAEG modules
try:
    from data_loader import DataLoader
    from graph_builder import TAEGGraphBuilder
    print("✅ TAEG modules imported successfully")
except ImportError as e:
    print(f"❌ Error importing TAEG modules: {e}")
    print("⚠️ Creating simple data loader for exploration...")

class SimpleDataLoader:
    """Simple data loader for XML file exploration"""
    
    def __init__(self, data_dir):
        self.data_dir = Path(data_dir)
        self.gospel_files = {
            'matthew': 'EnglishNIVMatthew40_PW.xml',
            'mark': 'EnglishNIVMark41_PW.xml',
            'luke': 'EnglishNIVLuke42_PW.xml',
            'john': 'EnglishNIVJohn43_PW.xml'
        }
        self.chronology_file = 'ChronologyOfTheFourGospels_PW.xml'
    
    def check_files(self):
        """Check if all files exist"""
        results = {}
        
        # Check chronology file
        chronology_path = self.data_dir / self.chronology_file
        results['chronology'] = {
            'exists': chronology_path.exists(),
            'path': chronology_path,
            'size': chronology_path.stat().st_size if chronology_path.exists() else 0
        }
        
        # Check gospel files
        results['gospels'] = {}
        for gospel, filename in self.gospel_files.items():
            filepath = self.data_dir / filename
            results['gospels'][gospel] = {
                'exists': filepath.exists(),
                'path': filepath,
                'size': filepath.stat().st_size if filepath.exists() else 0
            }
        
        return results

# Initialize data loader
data_dir = project_root / "data"
loader = SimpleDataLoader(data_dir)

# Check files
file_status = loader.check_files()

print("📊 Data files status:")
print(f"Chronology: {'✅' if file_status['chronology']['exists'] else '❌'} ({file_status['chronology']['size']:,} bytes)")

for gospel, info in file_status['gospels'].items():
    status = '✅' if info['exists'] else '❌'
    size = f"({info['size']:,} bytes)" if info['exists'] else "(file not found)"
    print(f"{gospel.title()}: {status} {size}")

In [ ]:
# Função para analisar a estrutura de um arquivo XML
def analyze_xml_structure(xml_path, max_depth=3, sample_size=5):
    """Analisa a estrutura de um arquivo XML"""
    if not xml_path.exists():
        return {"error": "Arquivo não encontrado"}
    
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()
        
        def get_element_info(element, depth=0):
            info = {
                'tag': element.tag,
                'attributes': dict(element.attrib),
                'text_length': len(element.text.strip()) if element.text else 0,
                'children_count': len(list(element)),
                'depth': depth
            }
            
            if depth < max_depth and len(list(element)) > 0:
                children = list(element)[:sample_size]  # Amostra dos primeiros filhos
                info['children_sample'] = [get_element_info(child, depth + 1) for child in children]
            
            return info
        
        structure = get_element_info(root)
        
        # Estatísticas gerais
        all_elements = list(root.iter())
        tags_count = Counter(elem.tag for elem in all_elements)
        
        return {
            'root_info': structure,
            'total_elements': len(all_elements),
            'unique_tags': len(tags_count),
            'tag_frequencies': dict(tags_count.most_common(10)),
            'file_size': xml_path.stat().st_size
        }
        
    except ET.ParseError as e:
        return {"error": f"Erro de parsing XML: {e}"}
    except Exception as e:
        return {"error": f"Erro inesperado: {e}"}

# Analisar estrutura do arquivo de cronologia
print("🔍 Analisando estrutura do arquivo de cronologia...")
chronology_analysis = analyze_xml_structure(file_status['chronology']['path'])

if 'error' not in chronology_analysis:
    print(f"📊 Elementos totais: {chronology_analysis['total_elements']}")
    print(f"🏷️ Tags únicas: {chronology_analysis['unique_tags']}")
    print(f"📈 Tags mais frequentes: {list(chronology_analysis['tag_frequencies'].keys())[:5]}")
    print(f"🔖 Tag raiz: {chronology_analysis['root_info']['tag']}")
    print(f"👶 Filhos da raiz: {chronology_analysis['root_info']['children_count']}")
else:
    print(f"❌ Erro: {chronology_analysis['error']}")

In [ ]:
# Analisar estrutura dos arquivos dos evangelhos
print("🔍 Analisando estrutura dos arquivos dos evangelhos...")

gospel_analyses = {}
for gospel, info in file_status['gospels'].items():
    if info['exists']:
        print(f"\n📖 Analisando {gospel.title()}...")
        analysis = analyze_xml_structure(info['path'])
        gospel_analyses[gospel] = analysis
        
        if 'error' not in analysis:
            print(f"   📊 Elementos: {analysis['total_elements']}")
            print(f"   🏷️ Tags únicas: {analysis['unique_tags']}")
            print(f"   📈 Tags principais: {list(analysis['tag_frequencies'].keys())[:3]}")
        else:
            print(f"   ❌ {analysis['error']}")

# Criar DataFrame para comparação
if gospel_analyses:
    comparison_data = []
    for gospel, analysis in gospel_analyses.items():
        if 'error' not in analysis:
            comparison_data.append({
                'Evangelho': gospel.title(),
                'Elementos_Totais': analysis['total_elements'],
                'Tags_Unicas': analysis['unique_tags'],
                'Tamanho_Arquivo_KB': analysis['file_size'] / 1024,
                'Tag_Raiz': analysis['root_info']['tag']
            })
    
    if comparison_data:
        df_gospels = pd.DataFrame(comparison_data)
        print("\n📊 Comparação dos evangelhos:")
        print(df_gospels.to_string(index=False))

## 3. Graph Construction from Chronology

Building the temporal event graph based on the chronology XML structure.

In [ ]:
# Função para extrair eventos da cronologia
def extract_chronology_events(xml_path):
    """Extrai eventos do arquivo de cronologia"""
    if not xml_path.exists():
        return []
    
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()
        
        events = []
        for event_elem in root.findall('.//event'):
            event_id = event_elem.get('id', '')
            if not event_id:
                continue
            
            # Extrair referências para cada evangelho
            event_data = {'id': event_id}
            gospels = ['matthew', 'mark', 'luke', 'john']
            
            for gospel in gospels:
                gospel_elem = event_elem.find(gospel)
                if gospel_elem is not None and gospel_elem.text:
                    event_data[gospel] = gospel_elem.text.strip()
                else:
                    event_data[gospel] = None
            
            events.append(event_data)
        
        return events
        
    except Exception as e:
        print(f"Erro ao extrair eventos: {e}")
        return []

# Extrair eventos da cronologia
print("📅 Extraindo eventos da cronologia...")
events = extract_chronology_events(file_status['chronology']['path'])

print(f"📊 Total de eventos extraídos: {len(events)}")

if events:
    # Analisar cobertura por evangelho
    gospel_coverage = {}
    for gospel in ['matthew', 'mark', 'luke', 'john']:
        count = sum(1 for event in events if event.get(gospel) is not None)
        gospel_coverage[gospel] = count
        print(f"   {gospel.title()}: {count} eventos ({count/len(events)*100:.1f}%)")
    
    # Mostrar alguns exemplos
    print(f"\n🔍 Primeiros 3 eventos como exemplo:")
    for i, event in enumerate(events[:3]):
        print(f"\nEvento {event['id']}:")
        for gospel in ['matthew', 'mark', 'luke', 'john']:
            ref = event.get(gospel, 'N/A')
            print(f"   {gospel.title()}: {ref}")
    
    # Criar DataFrame para análise
    events_df = pd.DataFrame(events)
    print(f"\n📊 Dimensões do DataFrame: {events_df.shape}")
else:
    print("❌ Nenhum evento foi extraído")

In [ ]:
# Construir grafo básico usando NetworkX
def build_basic_graph(events):
    """Constrói um grafo básico a partir dos eventos"""
    G = nx.DiGraph()
    
    # Adicionar nós (eventos)
    for event in events:
        event_id = event['id']
        participating_gospels = [g for g in ['matthew', 'mark', 'luke', 'john'] 
                               if event.get(g) is not None]
        
        G.add_node(event_id, 
                   participating_gospels=participating_gospels,
                   num_gospels=len(participating_gospels))
    
    # Adicionar arestas temporais para cada evangelho
    gospels = ['matthew', 'mark', 'luke', 'john']
    edge_count = 0
    
    for gospel in gospels:
        # Obter sequência de eventos para este evangelho
        gospel_events = [event['id'] for event in events if event.get(gospel) is not None]
        
        # Criar arestas sequenciais
        for i in range(len(gospel_events) - 1):
            current_event = gospel_events[i]
            next_event = gospel_events[i + 1]
            
            if G.has_edge(current_event, next_event):
                # Aresta já existe, adicionar este evangelho
                G[current_event][next_event]['gospels'].append(gospel)
                G[current_event][next_event]['weight'] += 1
            else:
                # Criar nova aresta
                G.add_edge(current_event, next_event, 
                          gospels=[gospel], weight=1)
                edge_count += 1
    
    return G, edge_count

if events:
    print("🕸️ Construindo grafo temporal...")
    graph, unique_edges = build_basic_graph(events)
    
    print(f"📊 Estatísticas do grafo:")
    print(f"   Nós (eventos): {graph.number_of_nodes()}")
    print(f"   Arestas únicas: {unique_edges}")
    print(f"   Arestas totais: {graph.number_of_edges()}")
    print(f"   Componentes conectados: {nx.number_weakly_connected_components(graph)}")
    
    # Analisar graus dos nós
    degrees = [d for n, d in graph.degree()]
    in_degrees = [d for n, d in graph.in_degree()]
    out_degrees = [d for n, d in graph.out_degree()]
    
    print(f"   Grau médio: {np.mean(degrees):.2f}")
    print(f"   Grau máximo: {max(degrees) if degrees else 0}")
    print(f"   In-degree médio: {np.mean(in_degrees):.2f}")
    print(f"   Out-degree médio: {np.mean(out_degrees):.2f}")
    
    # Nós por número de evangelhos
    gospels_distribution = {}
    for node in graph.nodes():
        num_gospels = graph.nodes[node]['num_gospels']
        gospels_distribution[num_gospels] = gospels_distribution.get(num_gospels, 0) + 1
    
    print(f"\n📖 Distribuição por número de evangelhos:")
    for num_gospels, count in sorted(gospels_distribution.items()):
        print(f"   {num_gospels} evangelho(s): {count} eventos ({count/len(events)*100:.1f}%)")

In [ ]:
# Visualização básica do grafo
if events and 'graph' in locals():
    print("📊 Criando visualizações do grafo...")
    
    # Configurar figura com subplots
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('TAEG - Análise do Grafo Temporal de Eventos', fontsize=16, fontweight='bold')
    
    # 1. Distribuição de graus
    degrees = [d for n, d in graph.degree()]
    axes[0, 0].hist(degrees, bins=20, alpha=0.7, color='skyblue', edgecolor='black')
    axes[0, 0].set_title('Distribuição de Graus dos Nós')
    axes[0, 0].set_xlabel('Grau')
    axes[0, 0].set_ylabel('Frequência')
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Participação por evangelho
    gospel_counts = [gospel_coverage[g] for g in ['matthew', 'mark', 'luke', 'john']]
    gospel_names = ['Mateus', 'Marcos', 'Lucas', 'João']
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']
    
    bars = axes[0, 1].bar(gospel_names, gospel_counts, color=colors, alpha=0.8)
    axes[0, 1].set_title('Eventos por Evangelho')
    axes[0, 1].set_ylabel('Número de Eventos')
    axes[0, 1].grid(True, alpha=0.3, axis='y')
    
    # Adicionar valores nas barras
    for bar, count in zip(bars, gospel_counts):
        axes[0, 1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
                        str(count), ha='center', va='bottom', fontweight='bold')
    
    # 3. Distribuição por número de evangelhos
    num_gospels_data = list(gospels_distribution.keys())
    num_gospels_counts = list(gospels_distribution.values())
    
    axes[1, 0].pie(num_gospels_counts, labels=[f'{n} Evang.' for n in num_gospels_data], 
                   autopct='%1.1f%%', startangle=90, colors=['#FFE66D', '#FF6B6B', '#4ECDC4', '#45B7D1'])
    axes[1, 0].set_title('Eventos por Número de Evangelhos')
    
    # 4. Componentes conectados (se houver múltiplos)
    components = list(nx.weakly_connected_components(graph))
    component_sizes = [len(comp) for comp in components]
    
    if len(components) > 1:
        axes[1, 1].bar(range(1, len(components) + 1), component_sizes, 
                       color='lightcoral', alpha=0.8)
        axes[1, 1].set_title('Tamanho dos Componentes Conectados')
        axes[1, 1].set_xlabel('Componente')
        axes[1, 1].set_ylabel('Número de Nós')
    else:
        axes[1, 1].text(0.5, 0.5, f'Grafo Conectado\\n{len(component_sizes[0])} nós', 
                        ha='center', va='center', transform=axes[1, 1].transAxes,
                        fontsize=14, bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgreen"))
        axes[1, 1].set_title('Conectividade do Grafo')
        axes[1, 1].set_xticks([])
        axes[1, 1].set_yticks([])
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Visualizações criadas com sucesso")

## 4. Node Content Extraction and Alignment

Extracting textual content for each event node from the four gospels and aligning them temporally.

In [ ]:
# Função para parsear referências de versículos
import re

def parse_verse_reference(ref_str):
    """Parse uma referência de versículo (ex: '2:1-12' ou '1:5')"""
    if not ref_str or ref_str.strip() == '':
        return []
    
    # Padrão para capturar capítulo:verso ou capítulo:verso-verso
    pattern = r'(\d+):(\d+)(?:-(\d+))?'
    matches = re.findall(pattern, ref_str.strip())
    
    references = []
    for match in matches:
        chapter = int(match[0])
        start_verse = int(match[1])
        end_verse = int(match[2]) if match[2] else start_verse
        
        references.append({
            'chapter': chapter,
            'start_verse': start_verse,
            'end_verse': end_verse
        })
    
    return references

# Função para extrair texto simplificado de um evangelho
def extract_gospel_text_simple(xml_path, target_chapter, target_verses):
    """Extração simplificada de texto de evangelho"""
    if not xml_path.exists():
        return ""
    
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()
        
        # Esta é uma implementação simplificada
        # Na prática, você precisaria adaptar para a estrutura real dos XMLs
        
        # Procurar elementos que contenham o texto
        texts = []
        for elem in root.iter():
            if elem.text and elem.text.strip():
                # Simplificado: assumir que encontramos texto relevante
                if len(elem.text.strip()) > 10:  # Filtrar textos muito curtos
                    texts.append(elem.text.strip())
        
        # Retornar uma amostra do texto (limitado para demonstração)
        if texts:
            return " ".join(texts[:3])  # Primeiros 3 textos encontrados
        
        return f"[Texto para capítulo {target_chapter}, versículos {target_verses} não encontrado]"
        
    except Exception as e:
        return f"[Erro ao extrair texto: {e}]"

# Analisar referências de versículos nos eventos
print("📖 Analisando referências de versículos...")

verse_analysis = {
    'total_references': 0,
    'by_gospel': {'matthew': 0, 'mark': 0, 'luke': 0, 'john': 0},
    'reference_patterns': [],
    'sample_extractions': []
}

if events:
    for i, event in enumerate(events[:10]):  # Analisar primeiros 10 eventos
        event_refs = {}
        
        for gospel in ['matthew', 'mark', 'luke', 'john']:
            ref_str = event.get(gospel)
            if ref_str:
                refs = parse_verse_reference(ref_str)
                event_refs[gospel] = refs
                verse_analysis['by_gospel'][gospel] += len(refs)
                verse_analysis['total_references'] += len(refs)
                
                # Coletar padrões de referência
                if refs:
                    verse_analysis['reference_patterns'].append({
                        'event_id': event['id'],
                        'gospel': gospel,
                        'reference': ref_str,
                        'parsed_count': len(refs)
                    })
        
        # Amostra de extração de texto (para demonstração)
        if i < 3:  # Apenas para os primeiros 3 eventos
            sample_text = {}
            for gospel in ['matthew', 'mark', 'luke', 'john']:
                if event.get(gospel) and gospel in file_status['gospels'] and file_status['gospels'][gospel]['exists']:
                    refs = event_refs.get(gospel, [])
                    if refs:
                        ref = refs[0]  # Primeira referência
                        text = extract_gospel_text_simple(
                            file_status['gospels'][gospel]['path'],
                            ref['chapter'],
                            range(ref['start_verse'], ref['end_verse'] + 1)
                        )
                        sample_text[gospel] = text[:200] + "..." if len(text) > 200 else text
            
            verse_analysis['sample_extractions'].append({
                'event_id': event['id'],
                'texts': sample_text
            })

print(f"📊 Resultados da análise:")
print(f"   Total de referências: {verse_analysis['total_references']}")
for gospel, count in verse_analysis['by_gospel'].items():
    print(f"   {gospel.title()}: {count} referências")

print(f"\n🔍 Padrões de referência (primeiros 5):")
for pattern in verse_analysis['reference_patterns'][:5]:
    print(f"   Evento {pattern['event_id']} - {pattern['gospel']}: {pattern['reference']} ({pattern['parsed_count']} refs)")

print(f"\n📝 Amostras de texto extraído:")
for sample in verse_analysis['sample_extractions']:
    print(f"\n   Evento {sample['event_id']}:")
    for gospel, text in sample['texts'].items():
        print(f"      {gospel.title()}: {text[:100]}{'...' if len(text) > 100 else ''}")

## 4. Temporal Edge Analysis

Detailed analysis of temporal edges in the TAEG graph and their relationships.

In [ ]:
# Análise detalhada das arestas temporais
def analyze_temporal_edges(graph, events):
    """Analisa as arestas temporais do grafo"""
    analysis = {
        'total_edges': graph.number_of_edges(),
        'edges_by_gospel': {'matthew': 0, 'mark': 0, 'luke': 0, 'john': 0},
        'edge_weights': [],
        'multi_gospel_edges': 0,
        'sequential_paths': {},
        'edge_details': []
    }
    
    # Analisar cada aresta
    for source, target, data in graph.edges(data=True):
        edge_gospels = data.get('gospels', [])
        edge_weight = data.get('weight', 1)
        
        analysis['edge_weights'].append(edge_weight)
        
        # Contar arestas por evangelho
        for gospel in edge_gospels:
            analysis['edges_by_gospel'][gospel] += 1
        
        # Arestas com múltiplos evangelhos
        if len(edge_gospels) > 1:
            analysis['multi_gospel_edges'] += 1
        
        # Detalhes das primeiras arestas
        if len(analysis['edge_details']) < 10:
            analysis['edge_details'].append({
                'source': source,
                'target': target,
                'gospels': edge_gospels,
                'weight': edge_weight
            })
    
    # Analisar caminhos sequenciais por evangelho
    for gospel in ['matthew', 'mark', 'luke', 'john']:
        gospel_events = [event['id'] for event in events if event.get(gospel) is not None]
        analysis['sequential_paths'][gospel] = {
            'length': len(gospel_events),
            'first_event': gospel_events[0] if gospel_events else None,
            'last_event': gospel_events[-1] if gospel_events else None,
            'sample_sequence': gospel_events[:5] if len(gospel_events) >= 5 else gospel_events
        }
    
    return analysis

if events and 'graph' in locals():
    print("🔗 Analisando arestas temporais...")
    edge_analysis = analyze_temporal_edges(graph, events)
    
    print(f"📊 Estatísticas das arestas:")
    print(f"   Total de arestas: {edge_analysis['total_edges']}")
    print(f"   Arestas com múltiplos evangelhos: {edge_analysis['multi_gospel_edges']}")
    print(f"   Peso médio das arestas: {np.mean(edge_analysis['edge_weights']):.2f}")
    
    print(f"\n📖 Arestas por evangelho:")
    for gospel, count in edge_analysis['edges_by_gospel'].items():
        print(f"   {gospel.title()}: {count} arestas")
    
    print(f"\n🔄 Caminhos sequenciais:")
    for gospel, path_info in edge_analysis['sequential_paths'].items():
        print(f"   {gospel.title()}: {path_info['length']} eventos")
        print(f"      Primeiro: {path_info['first_event']} → Último: {path_info['last_event']}")
        print(f"      Amostra: {' → '.join(path_info['sample_sequence'])}")
    
    print(f"\n🔍 Detalhes das primeiras arestas:")
    for detail in edge_analysis['edge_details'][:5]:
        gospels_str = ', '.join(detail['gospels'])
        print(f"   {detail['source']} → {detail['target']} (peso: {detail['weight']}, evangelhos: {gospels_str})")

In [ ]:
# Visualização interativa do grafo com Plotly
def create_interactive_graph_visualization(graph, max_nodes=50):
    """Cria visualização interativa do grafo usando Plotly"""
    
    # Limitar número de nós para visualização
    if graph.number_of_nodes() > max_nodes:
        # Pegar uma amostra dos nós mais conectados
        degree_centrality = nx.degree_centrality(graph)
        top_nodes = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:max_nodes]
        subgraph = graph.subgraph([node for node, _ in top_nodes])
    else:
        subgraph = graph
    
    # Calcular posições usando layout spring
    pos = nx.spring_layout(subgraph, k=1, iterations=50)
    
    # Preparar dados dos nós
    node_x = []
    node_y = []
    node_text = []
    node_colors = []
    
    for node in subgraph.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        
        # Informações do nó
        num_gospels = subgraph.nodes[node].get('num_gospels', 0)
        participating_gospels = subgraph.nodes[node].get('participating_gospels', [])
        
        node_text.append(f"Evento: {node}<br>Evangelhos: {num_gospels}<br>Detalhes: {', '.join(participating_gospels)}")
        
        # Cor baseada no número de evangelhos
        if num_gospels == 1:
            node_colors.append('lightblue')
        elif num_gospels == 2:
            node_colors.append('lightgreen')
        elif num_gospels == 3:
            node_colors.append('orange')
        else:
            node_colors.append('red')
    
    # Preparar dados das arestas
    edge_x = []
    edge_y = []
    edge_text = []
    
    for edge in subgraph.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
    
    # Criar traces
    edge_trace = go.Scatter(x=edge_x, y=edge_y,
                           line=dict(width=0.5, color='#888'),
                           hoverinfo='none',
                           mode='lines')
    
    node_trace = go.Scatter(x=node_x, y=node_y,
                           mode='markers',
                           hoverinfo='text',
                           text=node_text,
                           marker=dict(size=10,
                                      color=node_colors,
                                      line=dict(width=2, color='black')))
    
    # Criar figura
    fig = go.Figure(data=[edge_trace, node_trace],
                   layout=go.Layout(
                       title=f'TAEG - Grafo Temporal de Eventos (Amostra de {subgraph.number_of_nodes()} nós)',
                       titlefont_size=16,
                       showlegend=False,
                       hovermode='closest',
                       margin=dict(b=20,l=5,r=5,t=40),
                       annotations=[ dict(
                           text="Cores: Azul=1 evangelho, Verde=2, Laranja=3, Vermelho=4",
                           showarrow=False,
                           xref="paper", yref="paper",
                           x=0.005, y=-0.002,
                           xanchor="left", yanchor="bottom",
                           font=dict(color="black", size=12)
                       )],
                       xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                       yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))
    
    return fig

if events and 'graph' in locals():
    print("🎨 Criando visualização interativa do grafo...")
    try:
        interactive_fig = create_interactive_graph_visualization(graph)
        interactive_fig.show()
        print("✅ Visualização interativa criada! Passe o mouse sobre os nós para ver detalhes.")
    except Exception as e:
        print(f"❌ Erro ao criar visualização interativa: {e}")
        print("Continuando sem a visualização interativa...")

## 5. TAEG Model Implementation

In this section, we will implement and test the TAEG (Temporal Alignment Event Graph) model that was developed for the project.

In [ ]:
# Importar módulos do modelo TAEG
sys.path.append('../src')
from models import TAEGModel, GraphAttentionEncoder
from graph_builder import TAEGGraphBuilder

# Configuração do modelo
model_config = {
    'input_dim': 768,  # Dimensão dos embeddings BERT
    'hidden_dim': 256,
    'num_attention_heads': 8,
    'num_gat_layers': 2,
    'dropout': 0.1,
    'bart_model_name': 'facebook/bart-base'
}

print("🧠 Configuração do Modelo TAEG:")
print("=" * 50)
for key, value in model_config.items():
    print(f"{key}: {value}")

# Função para criar e inicializar o modelo
def create_taeg_model(config):
    """Cria e inicializa o modelo TAEG"""
    try:
        model = TAEGModel(
            input_dim=config['input_dim'],
            hidden_dim=config['hidden_dim'],
            num_attention_heads=config['num_attention_heads'],
            num_gat_layers=config['num_gat_layers'],
            dropout=config['dropout'],
            bart_model_name=config['bart_model_name']
        )
        return model
    except Exception as e:
        print(f"❌ Erro ao criar modelo: {e}")
        return None

# Criar o modelo
print("\n🔧 Criando modelo TAEG...")
try:
    taeg_model = create_taeg_model(model_config)
    if taeg_model:
        print("✅ Modelo TAEG criado com sucesso!")
        
        # Informações sobre o modelo
        total_params = sum(p.numel() for p in taeg_model.parameters())
        trainable_params = sum(p.numel() for p in taeg_model.parameters() if p.requires_grad)
        
        print(f"📊 Total de parâmetros: {total_params:,}")
        print(f"📊 Parâmetros treináveis: {trainable_params:,}")
        
        # Estrutura do modelo
        print("\n🏗️ Estrutura do modelo:")
        print(taeg_model)
    else:
        print("❌ Falha ao criar o modelo")
except Exception as e:
    print(f"❌ Erro durante criação do modelo: {e}")
    taeg_model = None

In [ ]:
# Teste de inferência com dados de exemplo
def test_model_inference(model, graph, sample_text="Jesus ensinou aos discípulos."):
    """Testa inferência do modelo com dados de exemplo"""
    
    if model is None or graph is None:
        print("❌ Modelo ou grafo não disponível para teste")
        return None
    
    try:
        # Converter grafo para formato PyTorch Geometric
        builder = TAEGGraphBuilder()
        torch_data = builder._convert_to_torch_geometric(graph)
        
        print(f"📊 Dados do grafo para inferência:")
        print(f"   - Número de nós: {torch_data.x.shape[0]}")
        print(f"   - Dimensão dos features: {torch_data.x.shape[1]}")
        print(f"   - Número de arestas: {torch_data.edge_index.shape[1]}")
        
        # Preparar input para o modelo
        model.eval()
        with torch.no_grad():
            # Simular embeddings de entrada (normalmente viriam do BERT)
            if torch_data.x.shape[1] != model_config['input_dim']:
                # Ajustar dimensão se necessário
                input_features = torch.randn(torch_data.x.shape[0], model_config['input_dim'])
            else:
                input_features = torch_data.x
            
            print(f"\n🔄 Executando inferência com texto: '{sample_text}'")
            
            # Forward pass através do encoder GAT
            gat_output = model.gat_encoder(input_features, torch_data.edge_index)
            print(f"✅ Saída do encoder GAT: {gat_output.shape}")
            
            # Simular geração de resumo
            sample_summary = model.generate_summary([sample_text], max_length=50)
            print(f"📝 Resumo gerado: {sample_summary[0] if sample_summary else 'Erro na geração'}")
            
            return gat_output, sample_summary
            
    except Exception as e:
        print(f"❌ Erro durante teste de inferência: {e}")
        return None

# Executar teste de inferência
if 'taeg_model' in locals() and taeg_model and 'graph' in locals() and graph:
    print("🧪 Testando inferência do modelo...")
    inference_result = test_model_inference(taeg_model, graph)
    
    if inference_result:
        print("✅ Teste de inferência concluído com sucesso!")
    else:
        print("❌ Teste de inferência falhou")
else:
    print("⚠️ Modelo ou grafo não disponível para teste de inferência")

## 6. Baseline Models Configuration

To compare TAEG performance, we will configure the baseline models: PEGASUS, PRIMERA and LexRank.

In [ ]:
# Importar modelos baseline
from models import PegasusBaseline, PrimeraBaseline, LexRankBaseline

# Configurações dos modelos baseline
baseline_configs = {
    'pegasus': {
        'model_name': 'google/pegasus-xsum',
        'max_length': 128,
        'min_length': 30
    },
    'primera': {
        'model_name': 'allenai/PRIMERA',
        'max_length': 256,
        'min_length': 50
    },
    'lexrank': {
        'sentence_count': 5,
        'threshold': 0.1
    }
}

print("📋 Configurações dos Modelos Baseline:")
print("=" * 60)
for model_name, config in baseline_configs.items():
    print(f"\n{model_name.upper()}:")
    for key, value in config.items():
        print(f"  {key}: {value}")

# Função para inicializar modelos baseline
def initialize_baseline_models():
    """Inicializa todos os modelos baseline"""
    models = {}
    
    try:
        print("\n🔧 Inicializando PEGASUS...")
        models['pegasus'] = PegasusBaseline(
            model_name=baseline_configs['pegasus']['model_name']
        )
        print("✅ PEGASUS inicializado")
    except Exception as e:
        print(f"❌ Erro ao inicializar PEGASUS: {e}")
        models['pegasus'] = None
    
    try:
        print("\n🔧 Inicializando PRIMERA...")
        models['primera'] = PrimeraBaseline(
            model_name=baseline_configs['primera']['model_name']
        )
        print("✅ PRIMERA inicializado")
    except Exception as e:
        print(f"❌ Erro ao inicializar PRIMERA: {e}")
        models['primera'] = None
    
    try:
        print("\n🔧 Inicializando LexRank...")
        models['lexrank'] = LexRankBaseline()
        print("✅ LexRank inicializado")
    except Exception as e:
        print(f"❌ Erro ao inicializar LexRank: {e}")
        models['lexrank'] = None
    
    return models

# Inicializar modelos baseline
print("🚀 Inicializando modelos baseline...")
baseline_models = initialize_baseline_models()

# Verificar quais modelos foram inicializados com sucesso
available_models = [name for name, model in baseline_models.items() if model is not None]
failed_models = [name for name, model in baseline_models.items() if model is None]

print(f"\n📊 Resumo da inicialização:")
print(f"✅ Modelos disponíveis: {', '.join(available_models) if available_models else 'Nenhum'}")
print(f"❌ Modelos com falha: {', '.join(failed_models) if failed_models else 'Nenhum'}")

In [ ]:
# Teste rápido dos modelos baseline
def test_baseline_models(models, sample_texts):
    """Testa rapidamente os modelos baseline com textos de exemplo"""
    
    if not sample_texts:
        sample_texts = [
            "Jesus ensinou aos discípulos sobre o Reino de Deus.",
            "Os apóstolos seguiram Jesus durante sua jornada.",
            "A mensagem de amor e perdão foi central nos ensinamentos."
        ]
    
    results = {}
    
    for model_name, model in models.items():
        if model is None:
            print(f"⚠️ {model_name.upper()} não disponível para teste")
            continue
            
        try:
            print(f"\n🧪 Testando {model_name.upper()}...")
            
            if model_name == 'lexrank':
                # LexRank precisa de múltiplos documentos
                summary = model.summarize(sample_texts)
            else:
                # PEGASUS e PRIMERA podem trabalhar com texto concatenado
                combined_text = " ".join(sample_texts)
                summary = model.summarize([combined_text])
                summary = summary[0] if summary else "Erro na geração"
            
            results[model_name] = summary
            print(f"📝 Resumo gerado: {summary[:100]}{'...' if len(summary) > 100 else ''}")
            
        except Exception as e:
            print(f"❌ Erro ao testar {model_name}: {e}")
            results[model_name] = None
    
    return results

# Preparar textos de exemplo dos evangelhos
sample_gospel_texts = []
if events:
    # Extrair alguns textos dos eventos carregados
    sample_events = list(events.keys())[:3]
    for event_name in sample_events:
        verses = data_loader.get_verses_for_event(event_name)
        if verses:
            sample_texts = [verse.text for verse in verses[:2]]  # Primeiros 2 versos
            sample_gospel_texts.extend(sample_texts)

if not sample_gospel_texts:
    sample_gospel_texts = [
        "E Jesus, vendo as multidões, subiu ao monte; e, assentando-se, aproximaram-se dele os seus discípulos.",
        "E, abrindo a sua boca, os ensinava, dizendo: Bem-aventurados os pobres de espírito, porque deles é o reino dos céus.",
        "Bem-aventurados os que choram, porque eles serão consolados."
    ]

print("📚 Textos de exemplo para teste:")
for i, text in enumerate(sample_gospel_texts[:3], 1):
    print(f"{i}. {text[:80]}{'...' if len(text) > 80 else ''}")

# Executar testes dos modelos baseline
if baseline_models:
    print("\n🧪 Executando testes dos modelos baseline...")
    baseline_results = test_baseline_models(baseline_models, sample_gospel_texts)
    
    print(f"\n📊 Resumo dos resultados:")
    for model_name, result in baseline_results.items():
        status = "✅ Sucesso" if result else "❌ Falha"
        print(f"{model_name.upper()}: {status}")
else:
    print("⚠️ Nenhum modelo baseline disponível para teste")

## 7. Training Pipeline

Configuration and testing of the training pipeline for the TAEG model.

In [ ]:
# Importar módulos de treinamento
from train import TAEGTrainer, TrainingConfig, EarlyStopping

# Configuração de treinamento
training_config = TrainingConfig(
    batch_size=8,
    learning_rate=1e-4,
    num_epochs=10,
    warmup_steps=100,
    max_grad_norm=1.0,
    early_stopping_patience=3,
    save_every_n_epochs=2
)

print("🎯 Configuração de Treinamento:")
print("=" * 50)
print(f"Batch Size: {training_config.batch_size}")
print(f"Learning Rate: {training_config.learning_rate}")
print(f"Número de Épocas: {training_config.num_epochs}")
print(f"Warmup Steps: {training_config.warmup_steps}")
print(f"Max Grad Norm: {training_config.max_grad_norm}")
print(f"Early Stopping Patience: {training_config.early_stopping_patience}")
print(f"Salvar a cada N épocas: {training_config.save_every_n_epochs}")

# Função para preparar dados de treinamento
def prepare_training_data(events, data_loader, train_ratio=0.8):
    """Prepara dados para treinamento dividindo em train/val"""
    
    if not events:
        print("❌ Nenhum evento disponível para treinamento")
        return None, None
    
    # Converter eventos em pares (input, target)
    training_pairs = []
    
    for event_name, event_data in events.items():
        verses = data_loader.get_verses_for_event(event_name)
        if verses and len(verses) >= 2:
            # Usar versos como input e criar um resumo simples como target
            input_texts = [verse.text for verse in verses]
            # Target simplificado (para demonstração)
            target_summary = f"Resumo do evento {event_name}: {verses[0].text[:50]}..."
            
            training_pairs.append({
                'input_texts': input_texts,
                'target_summary': target_summary,
                'event_name': event_name
            })
    
    if not training_pairs:
        print("❌ Nenhum par de treinamento criado")
        return None, None
    
    # Dividir em treino e validação
    split_idx = int(len(training_pairs) * train_ratio)
    train_data = training_pairs[:split_idx]
    val_data = training_pairs[split_idx:]
    
    print(f"📊 Dados preparados:")
    print(f"   Total de pares: {len(training_pairs)}")
    print(f"   Treinamento: {len(train_data)}")
    print(f"   Validação: {len(val_data)}")
    
    return train_data, val_data

# Preparar dados de treinamento
if events and data_loader:
    print("\n📦 Preparando dados de treinamento...")
    train_data, val_data = prepare_training_data(events, data_loader)
    
    if train_data and val_data:
        print("✅ Dados de treinamento preparados com sucesso!")
        
        # Mostrar exemplo de dados
        print(f"\n📋 Exemplo de dado de treinamento:")
        example = train_data[0]
        print(f"Evento: {example['event_name']}")
        print(f"Textos de entrada: {len(example['input_texts'])} versos")
        print(f"Primeiro verso: {example['input_texts'][0][:100]}...")
        print(f"Resumo alvo: {example['target_summary']}")
    else:
        print("❌ Falha ao preparar dados de treinamento")
else:
    print("⚠️ Eventos ou data_loader não disponíveis")

In [ ]:
# Configurar trainer e executar treinamento de demonstração
def setup_trainer(model, graph, config):
    """Configura o trainer para o modelo TAEG"""
    
    if model is None or graph is None:
        print("❌ Modelo ou grafo não disponível")
        return None
    
    try:
        trainer = TAEGTrainer(
            model=model,
            config=config,
            device='cpu'  # Usar CPU para demonstração
        )
        
        # Converter grafo para formato PyTorch Geometric
        builder = TAEGGraphBuilder()
        torch_data = builder._convert_to_torch_geometric(graph)
        
        print(f"✅ Trainer configurado com sucesso")
        print(f"📊 Dados do grafo: {torch_data.x.shape[0]} nós, {torch_data.edge_index.shape[1]} arestas")
        
        return trainer, torch_data
        
    except Exception as e:
        print(f"❌ Erro ao configurar trainer: {e}")
        return None

# Simular uma época de treinamento
def simulate_training_epoch(trainer, torch_data, train_data):
    """Simula uma época de treinamento (apenas para demonstração)"""
    
    if not trainer or not torch_data or not train_data:
        print("❌ Componentes necessários não disponíveis")
        return None
    
    try:
        print("🔄 Simulando época de treinamento...")
        
        # Em um treinamento real, isso seria feito com batches
        sample_batch = train_data[:min(2, len(train_data))]  # Pequeno batch para demo
        
        # Simular métricas de treinamento
        simulated_metrics = {
            'loss': 2.5 + np.random.normal(0, 0.1),
            'learning_rate': training_config.learning_rate,
            'grad_norm': np.random.uniform(0.5, 1.5)
        }
        
        print(f"📊 Métricas simuladas:")
        for metric, value in simulated_metrics.items():
            print(f"   {metric}: {value:.4f}")
        
        return simulated_metrics
        
    except Exception as e:
        print(f"❌ Erro durante simulação: {e}")
        return None

# Configurar e testar o trainer
if 'taeg_model' in locals() and taeg_model and 'graph' in locals() and graph:
    print("🔧 Configurando trainer...")
    trainer_setup = setup_trainer(taeg_model, graph, training_config)
    
    if trainer_setup and 'train_data' in locals() and train_data:
        trainer, torch_data = trainer_setup
        
        print("\n🧪 Executando simulação de treinamento...")
        training_metrics = simulate_training_epoch(trainer, torch_data, train_data)
        
        if training_metrics:
            print("✅ Simulação de treinamento concluída!")
            
            # Simular histórico de treinamento
            print(f"\n📈 Histórico simulado de 3 épocas:")
            for epoch in range(1, 4):
                loss = 2.5 - (epoch * 0.3) + np.random.normal(0, 0.1)
                print(f"Época {epoch}: Loss = {loss:.4f}")
        else:
            print("❌ Falha na simulação de treinamento")
    else:
        print("❌ Falha ao configurar trainer ou dados não disponíveis")
else:
    print("⚠️ Modelo TAEG ou grafo não disponíveis para teste de treinamento")

## 8. Evaluation and Metrics

Configuration and testing of the evaluation system with ROUGE, BERTScore and temporal coherence metrics.

In [ ]:
# Importar módulos de avaliação
from evaluate import ROUGEEvaluator, BERTScoreEvaluator, TemporalCoherenceEvaluator

# Configurar avaliadores
print("🎯 Configurando avaliadores...")

evaluators = {}

try:
    print("🔧 Inicializando ROUGE Evaluator...")
    evaluators['rouge'] = ROUGEEvaluator()
    print("✅ ROUGE Evaluator inicializado")
except Exception as e:
    print(f"❌ Erro ao inicializar ROUGE: {e}")
    evaluators['rouge'] = None

try:
    print("🔧 Inicializando BERTScore Evaluator...")
    evaluators['bertscore'] = BERTScoreEvaluator()
    print("✅ BERTScore Evaluator inicializado")
except Exception as e:
    print(f"❌ Erro ao inicializar BERTScore: {e}")
    evaluators['bertscore'] = None

try:
    print("🔧 Inicializando Temporal Coherence Evaluator...")
    evaluators['temporal'] = TemporalCoherenceEvaluator()
    print("✅ Temporal Coherence Evaluator inicializado")
except Exception as e:
    print(f"❌ Erro ao inicializar Temporal Coherence: {e}")
    evaluators['temporal'] = None

# Verificar avaliadores disponíveis
available_evaluators = [name for name, eval in evaluators.items() if eval is not None]
print(f"\n📊 Avaliadores disponíveis: {', '.join(available_evaluators) if available_evaluators else 'Nenhum'}")

# Função para testar avaliação
def test_evaluation_metrics(evaluators, predictions, references):
    """Testa métricas de avaliação com dados de exemplo"""
    
    results = {}
    
    for eval_name, evaluator in evaluators.items():
        if evaluator is None:
            print(f"⚠️ {eval_name.upper()} não disponível")
            continue
        
        try:
            print(f"\n🧪 Testando {eval_name.upper()}...")
            
            if eval_name == 'rouge':
                scores = evaluator.compute_rouge(predictions, references)
                results[eval_name] = scores
                print(f"📊 ROUGE-1: {scores.get('rouge1', 'N/A')}")
                print(f"📊 ROUGE-2: {scores.get('rouge2', 'N/A')}")
                print(f"📊 ROUGE-L: {scores.get('rougeL', 'N/A')}")
                
            elif eval_name == 'bertscore':
                scores = evaluator.compute_bertscore(predictions, references)
                results[eval_name] = scores
                if scores:
                    print(f"📊 BERTScore F1: {np.mean(scores.get('f1', [0])):.4f}")
                    print(f"📊 BERTScore Precision: {np.mean(scores.get('precision', [0])):.4f}")
                    print(f"📊 BERTScore Recall: {np.mean(scores.get('recall', [0])):.4f}")
                
            elif eval_name == 'temporal':
                # Para coerência temporal, precisamos de dados temporais
                if 'graph' in locals() and graph:
                    scores = evaluator.evaluate_temporal_coherence(predictions, graph)
                    results[eval_name] = scores
                    print(f"📊 Coerência Temporal: {scores.get('coherence_score', 'N/A')}")
                else:
                    print("⚠️ Grafo não disponível para avaliação temporal")
                    
        except Exception as e:
            print(f"❌ Erro ao testar {eval_name}: {e}")
            results[eval_name] = None
    
    return results

# Preparar dados de exemplo para avaliação
sample_predictions = [
    "Jesus ensinou aos discípulos sobre o amor e o perdão, mostrando o caminho da salvação.",
    "Os apóstolos seguiram Jesus em sua jornada, aprendendo com seus ensinamentos e milagres.",
    "A mensagem de esperança e fé foi central na pregação de Jesus aos povos."
]

sample_references = [
    "Jesus Cristo ensinou sobre o amor divino e o perdão, revelando o caminho para a vida eterna.",
    "Os doze apóstolos acompanharam Jesus durante seu ministério, sendo testemunhas de seus milagres.",
    "A proclamação da boa nova trouxe esperança e fé para todos os que ouviram a palavra de Deus."
]

print(f"\n📚 Dados de exemplo para avaliação:")
print(f"Predições: {len(sample_predictions)} resumos")
print(f"Referências: {len(sample_references)} resumos")

# Executar testes de avaliação
if evaluators:
    print("\n🧪 Executando testes de avaliação...")
    evaluation_results = test_evaluation_metrics(evaluators, sample_predictions, sample_references)
    
    print(f"\n📈 Resumo da Avaliação:")
    for eval_name, result in evaluation_results.items():
        status = "✅ Sucesso" if result else "❌ Falha"
        print(f"{eval_name.upper()}: {status}")
        
    # Calcular score médio (se disponível)
    if evaluation_results.get('rouge') and evaluation_results.get('bertscore'):
        rouge_l = evaluation_results['rouge'].get('rougeL', 0)
        bert_f1 = np.mean(evaluation_results['bertscore'].get('f1', [0]))
        avg_score = (rouge_l + bert_f1) / 2
        print(f"\n🎯 Score médio (ROUGE-L + BERTScore): {avg_score:.4f}")
        
else:
    print("⚠️ Nenhum avaliador disponível")

In [ ]:
# Final exploration report
print("📋 TAEG DATA EXPLORATION FINAL REPORT")
print("=" * 80)

# Data status
if 'data_loader' in locals() and data_loader:
    print(f"✅ DataLoader: Configured and functional")
    if 'events' in locals() and events:
        print(f"✅ Events loaded: {len(events)} events")
    else:
        print(f"⚠️ Events: Not loaded (XML files required)")
else:
    print(f"❌ DataLoader: Not configured")

# Graph status
if 'graph' in locals() and graph:
    print(f"✅ TAEG Graph: {graph.number_of_nodes()} nodes, {graph.number_of_edges()} edges")
else:
    print(f"❌ TAEG Graph: Not built")

# Model status
if 'taeg_model' in locals() and taeg_model:
    total_params = sum(p.numel() for p in taeg_model.parameters())
    print(f"✅ TAEG Model: Configured ({total_params:,} parameters)")
else:
    print(f"❌ TAEG Model: Not configured")

if 'baseline_models' in locals():
    available_baselines = [name for name, model in baseline_models.items() if model is not None]
    print(f"✅ Baseline Models: {', '.join(available_baselines) if available_baselines else 'None'}")
else:
    print(f"❌ Baseline Models: Not configured")

# Training status
if 'training_config' in locals():
    print(f"✅ Training Configuration: Defined")
    if 'train_data' in locals() and train_data:
        print(f"✅ Training Data: {len(train_data)} samples")
    else:
        print(f"⚠️ Training Data: Not prepared")
else:
    print(f"❌ Training Configuration: Not defined")

# Evaluation status
if 'evaluators' in locals():
    available_evaluators = [name for name, eval in evaluators.items() if eval is not None]
    print(f"✅ Evaluators: {', '.join(available_evaluators) if available_evaluators else 'None'}")
else:
    print(f"❌ Evaluators: Not configured")

print("\n" + "=" * 80)
print("🎯 RECOMMENDED NEXT STEPS:")
print("1. Add gospel XML files to data/ directory")
print("2. Execute complete pipeline with real data")
print("3. Train TAEG model with gospel data")
print("4. Compare performance with baseline models")
print("5. Evaluate quality of generated summaries")
print("6. Detailed analysis of temporal coherence")

print("\n💡 TIPS:")
print("- Use 'python main.py --help' to see command line options")
print("- Configure GPU if available for faster training")
print("- Adjust hyperparameters based on initial results")
print("- Monitor validation metrics to avoid overfitting")

print("\n🔬 This notebook provides a solid foundation for TAEG project exploration and development!")
print("=" * 80)